## Kyle Demers
## Databases - Fall 2022
### Assignment 2 - Due Thursday, Oct 6 by midnight

**Make sure to submit your completed notebook as a PDF with all code and output legible**  
.ipynb, .py, or any other format files will not be graded!

Using the same NYC Health Inspection database we've been working on in class, write some SQL queries to answer the following questions.

**To help us grade, please pay attention to how the results should be ordered.  We may have to deduct points if you do not!**

---

In [39]:
# Libaray imports, database connections, etc here.
import pandas as pd
import sqlite3
conn = sqlite3.connect('test.db')
curs = conn.cursor()
df = pd.read_csv('nyc_rest_health_inspections.csv')

---
1) Group all restaurants by their DBA, and report the average number of violations per location.

For example, if there were 100 restaurants with the name "STARBUCKS", and they had a total of 1000 violations, then the average number of violations per location would be 1000/100=10.

Have your query return 4 fields:
- dba
- number of locations
- number of violations
- number of violations per location

Order the results with the largest number of locations on top

_Note: You will need to watch out for "integer division".  When you divide two numbers in SQL, if both of them are integers, then your answer will be missing it's fractional portion._

_Below is a simple example of this, as well as how to fix it. In short, you just need to make sure that the first value before the division is not an integer._

In [1]:
#Creating a temporary "in-memory" database for the purposes of demonstration
import sqlite3, pandas as pd
temp = sqlite3.connect(':memory:')

#Run a query dividing two integers and note the output is incorrect
pd.read_sql("SELECT 3/2;", temp)

,3/2
0,1


In [2]:
# Several ways to fix this.  Using the least amount of typing:
pd.read_sql("SELECT 1.*3/2;", temp)

,1.*3/2
0,1.5


In [ ]:
# A little less weird looking:
#pd.read_sql("SELECT 1.0*3/2;", temp)

In [ ]:
# More explicit, but more typing:
#pd.read_sql("SELECT CAST(3 as REAL)/2;", temp)

In [ ]:
# Essentially the same as above:
#pd.read_sql("SELECT CAST(3 as FLOAT)/2;", temp)

In [ ]:
# Closing the temporary database
#temp.close()

In [2]:
pd.read_sql('''
SELECT dba,NumLocations,NumViolations, CAST(NumViolations as Float)/NumLocations as AvgViolationsPerLocation
FROM(
    SELECT dba, count(camis) as NumLocations,NumViolations
    FROM (
            SELECT camis,DBA,count([violation code]) as NumViolations
            FROM test_table
            GROUP BY camis
            ORDER BY NumViolations DESC
         ) 
    GROUP BY dba
    )
GROUP BY dba
ORDER BY NumLocations DESC
;''',conn)

,dba,NumLocations,NumViolations,AvgViolationsPerLocation
0,DUNKIN' DONUTS,410,50,0.121951
1,SUBWAY,352,44,0.125000
2,STARBUCKS,281,24,0.085409
3,MCDONALD'S,206,36,0.174757
4,"DUNKIN' DONUTS, BASKIN ROBBINS",110,35,0.318182
...,...,...,...,...
20242,$1 PIZZA,1,8,8.000000
20243,#1 SABOR LATINO RESTAURANT,1,55,55.000000
20244,#1 Natural Juice Bar,1,6,6.000000
20245,#1 GARDEN CHINESE,1,27,27.000000


---
2) Get a list of all restaurants (by camis) who have been cited for a violation containing the words 'vermin', 'mice', 'roach', or 'rats'.

Have your query return the following columns:
- camis
- dba
- violation code
- violation description
- number of violations

Order the results with the most violations on top

In [3]:
pd.read_sql('''
SELECT camis, dba, [violation code],[violation description], count([violation code]) as NumViol
FROM test_table
WHERE [violation description] LIKE '%vermin%' 
OR [violation description] LIKE '%mice%' 
OR [violation description] LIKE '%roach%' 
OR [violation description] LIKE '%rats%' 
GROUP BY camis
ORDER BY NumViol Desc
;''',conn)

,CAMIS,DBA,VIOLATION CODE,VIOLATION DESCRIPTION,NumViol
0,50033122,PARTY WELL REST & ORIENTAL BAKERY,04M,Live roaches present in facility's food and/or...,34
1,40861669,NEW VICTORY RESTAURANT,04K,Evidence of rats or live rats present in facil...,34
2,41630632,J J NOODLE,04L,Evidence of mice or live mice present in facil...,32
3,41459659,T. K. KITCHEN,08A,Facility not vermin proof. Harborage or condit...,31
4,41541625,LI'S GOLDEN CITY RESTAURANT,04L,Evidence of mice or live mice present in facil...,30
...,...,...,...,...,...
16326,40364404,POLISH NATIONAL HOME,08A,Facility not vermin proof. Harborage or condit...,1
16327,40363427,BAGELS N BUNS,08A,Facility not vermin proof. Harborage or condit...,1
16328,40363298,CAFE METRO,08A,Facility not vermin proof. Harborage or condit...,1
16329,40359705,NATHAN'S FAMOUS,08A,Facility not vermin proof. Harborage or condit...,1


---
3) For each different CUISINE DESCRIPTION, determine the most commonly cited violation

Have your query return:

- cuisine description
- violation code
- violation description
- number of violations

Order the results alphabetically by the cuisine type.

_Hints:_
 - SQL has a 'max' function that works just like 'count' and 'avg'
 - If you're unsure of your results, just pick a cuisine type or two and write some more detailed queries to help validate your results 

In [35]:
pd.read_sql('''
SELECT [cuisine description], [violation code], [violation description], MAX(numviol) as [number of violations]
FROM (
    SELECT [violation code], [violation description], [cuisine description], count([violation code]) as numviol
    FROM test_table
    GROUP BY [violation code], [cuisine description]
    ORDER BY numviol DESC
    )
GROUP BY ([cuisine description])
;''',conn)

,cuisine description,violation code,violation description,number of violations
0,Afghan,10F,Non-food contact surface improperly constructe...,36
1,African,08A,Facility not vermin proof. Harborage or condit...,189
2,American,10F,Non-food contact surface improperly constructe...,13176
3,Armenian,10F,Non-food contact surface improperly constructe...,68
4,Asian,10F,Non-food contact surface improperly constructe...,741
...,...,...,...,...
79,Tex-Mex,10F,Non-food contact surface improperly constructe...,334
80,Thai,10F,Non-food contact surface improperly constructe...,648
81,Turkish,10F,Non-food contact surface improperly constructe...,162
82,Vegetarian,10F,Non-food contact surface improperly constructe...,218


In [37]:
### TEST
#pd.read_sql('''
#SELECT [violation code], [violation description], [cuisine description], count([violation code]) as numviol
#FROM test_table
#WHERE [cuisine description] LIKE 'vegetarian'
#GROUP BY [violation code], [cuisine description]
#ORDER BY numviol DESC
#;''',conn)

---
4) When a violation is cited, the action taken might not always be the same.

Count up all the combinations of violations and actions in the data.  Have your query return:

- violation
- violation description
- action
- number of violations

Order the results with the largest number of violations on top.

In [20]:
pd.read_sql('''
SELECT [violation code], [violation description], action, count([violation code]) as [number of violations]
FROM test_table
GROUP BY [violation code], [action]
ORDER BY [number of violations] DESC
;''',conn)
#need to number of violations for each violation *by action*

,VIOLATION CODE,VIOLATION DESCRIPTION,ACTION,number of violations
0,10F,Non-food contact surface improperly constructe...,Violations were cited in the following area(s).,54039
1,08A,Facility not vermin proof. Harborage or condit...,Violations were cited in the following area(s).,37990
2,02G,Cold food item held above 41Âº F (smoked fish ...,Violations were cited in the following area(s).,27142
3,04L,Evidence of mice or live mice present in facil...,Violations were cited in the following area(s).,26431
4,06D,"Food contact surface not properly washed, rins...",Violations were cited in the following area(s).,25683
...,...,...,...,...
239,18F,Permit not conspicuously displayed.,Establishment Closed by DOHMH. Violations wer...,1
240,20A,Food allergy information poster not conspicuou...,Establishment re-opened by DOHMH,1
241,20B,Food allergy information poster not posted in ...,No violations were recorded at the time of thi...,1
242,20E,Letter Grade or Grade Pending card not conspic...,Establishment Closed by DOHMH. Violations wer...,1


In [38]:
### TEST
#pd.read_sql('''
#SELECT *
#FROM(
#    SELECT [violation code], [violation description], action, count([action]) as [number of violations]
#    FROM test_table
#    GROUP BY [violation code], [action]
#    ORDER BY [number of violations] DESC
#    )
#WHERE [violation code] LIKE '10F'
#;''',conn)

---
5) How many restaurants (by camis) have been cited for 2 or more different violation codes, where they were cited for each of those violations more than 10 times?

Have your query return those restaurants and violations (just the ones they were cited for more than 10 times), and include the following columns:

- camis
- dba
- violation code
- violation description
- number of violations

Order the results alphabetically by DBA

In [44]:
pd.read_sql('''
WITH tnumviols AS
(SELECT camis, dba, [violation code], [violation description], count([violation code]) as numviol
FROM test_table
GROUP BY camis, [violation code]
),
tTotalviols AS
(SELECT count([numviol]) as totalViolsOver10, dba
FROM tnumviols
WHERE numviol >10
GROUP BY dba
)
SELECT dba , camis, [violation code], [violation description], numviol
From tTotalviols
JOIN tnumviols
USING(dba)
WHERE numviol > 10
and totalViolsOver10 > 1
;''',conn)

,dba,camis,violation code,violation description,numviol
0,218 RESTAURANT,41510846,02G,Cold food item held above 41Âº F (smoked fish ...,11
1,218 RESTAURANT,41510846,08A,Facility not vermin proof. Harborage or condit...,11
2,A-WAH RESTAURANT,41475257,04M,Live roaches present in facility's food and/or...,12
3,A-WAH RESTAURANT,41475257,08A,Facility not vermin proof. Harborage or condit...,12
4,B BO SING BAKERY,41692971,04L,Evidence of mice or live mice present in facil...,11
5,B BO SING BAKERY,41692971,08A,Facility not vermin proof. Harborage or condit...,11
6,CAFE PASHTRIKU,41719174,15J,Ashtray present in smoke-free area.,11
7,CAFE PASHTRIKU,41719174,15K,Operator failed to make good faith effort to i...,12
8,LATINO'S BAR & GRILL RESTAURANT,41164793,04N,Filth flies or food/refuse/sewage-associated (...,11
9,LATINO'S BAR & GRILL RESTAURANT,41164793,08A,Facility not vermin proof. Harborage or condit...,11


In [45]:
# Don't forget to close the database!
conn.close()

-- no help recieved from outside sources